
## Passwords Auditor with contraints


In [14]:

## !pip install zxcvbn


In [15]:


import string
import torch
import torch.nn as nn
import torch.nn.functional as F


In [16]:


VOCAB = list(string.ascii_letters + string.digits + "!@#$%^&*")
V     = len(VOCAB)
V


70

In [17]:

MASK_UPPER = torch.tensor([c in string.ascii_uppercase for c in VOCAB]).float()
MASK_LOWER = torch.tensor([c in string.ascii_lowercase for c in VOCAB]).float()
MASK_DIGIT = torch.tensor([c in string.digits          for c in VOCAB]).float()
MASK_SPEC  = torch.tensor([c in "!@#$%^&*"             for c in VOCAB]).float()

print(MASK_SPEC.shape)

MASK_SPEC


torch.Size([70])


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:


class TinyDecoder(nn.Module):
    
    def __init__(self, z_dim=128, T=12, V=V):
        super().__init__()
        self.T = T
        self.V = V
        
        self.net = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, T * V)
        )

    def forward(self, z):
        logits = self.net(z)                  # [B, T*V]
        
        return logits.view(z.size(0), self.T, self.V)  # [B, T, V]



In [19]:


def st_gumbel_softmax(logits, tau):
    
    eps = 1e-9
    
    g = -torch.log(-torch.log(torch.rand_like(logits) + eps) + eps)
    
    y_soft = F.softmax((logits + g) / tau, dim=-1)          # [B, T, V]
    
    y_hard = F.one_hot(
                 y_soft.argmax(-1), 
                 y_soft.size(-1)
    ).float()
    
    return y_hard + (y_soft - y_soft.detach())              # hard forward, soft grad


In [20]:


def class_present(p_pos):
    
    # p_pos: [B, T] probability per position of being in the class
    
    return 1.0 - torch.prod(1.0 - p_pos + 1e-6, dim=1)      # [B]



In [21]:


def entropy_bits_per_string(y_probs):
    # y_probs: [B, T, V], probs per token
    p = y_probs.clamp_min(1e-9)
    H_t = -(p * p.log()).sum(dim=-1)                         # nats, [B, T]
    H = H_t.mean(dim=1)                                      # [B]
    return H / torch.log(torch.tensor(2.0))                  # bits


In [22]:

# --- Decode one-hot/probabilities to strings ---

def decode(y_probs):
    idx = y_probs.argmax(dim=-1).cpu()                       # [B, T]
    out = []
    for row in idx.tolist():
        out.append("".join(VOCAB[i] for i in row))
    return out


In [23]:


def generate_passwords(batch=16, steps=200, T=12, H_min_bits=3.0, seed=0):
    torch.manual_seed(seed)

    model = TinyDecoder(T=T)
    for p in model.parameters():
        p.requires_grad = False                              # freeze decoder (we only optimize z)

    z = torch.randn(batch, 128, requires_grad=True)
    
    opt = torch.optim.Adam([z], lr=0.05)
    y_probs = None

    for t in range(steps):
        tau = max(0.8 - 0.003 * t, 0.2)

        logits = model(z)                                    # [B, T, V]
        
        y_probs = st_gumbel_softmax(logits, tau)             # [B, T, V]

        pU = (y_probs * MASK_UPPER).sum(dim=-1)              # [B, T]
        pL = (y_probs * MASK_LOWER).sum(dim=-1)
        pD = (y_probs * MASK_DIGIT).sum(dim=-1)
        pS = (y_probs * MASK_SPEC ).sum(dim=-1)

        need_upper = F.relu(1.0 - class_present(pU)).mean()
        need_lower = F.relu(1.0 - class_present(pL)).mean()
        need_digit = F.relu(1.0 - class_present(pD)).mean()
        need_spec  = F.relu(1.0 - class_present(pS)).mean()
        
        class_loss = need_upper + need_lower + need_digit + need_spec

        H_bits       = entropy_bits_per_string(  y_probs  )            # [B]
        entropy_loss = F.relu(H_min_bits - H_bits).mean()

        loss = class_loss + entropy_loss

        opt.zero_grad()
        loss.backward()
        opt.step()

    return decode(  y_probs.detach()  )





In [24]:

sample = generate_passwords(batch=8, steps=200, T=12, H_min_bits=3.0)

print("Sample:", sample)

constraintsGeneratedPasswords = sample


Sample: ['z4SvfVncfS14', 'o$^&^k&73Gs1', 'd6fTX6vI^!sy', '@A#UbUW7S$pz', 'Sf5I@n2$dejM', 'Vz$N1D1$xTPP', '2TR*pw&@KfJJ', '!PLcQ8xUdM%^']



## zxcvbn test tool


In [25]:

from zxcvbn import zxcvbn



In [26]:

result = zxcvbn("MyP@ssw0rd123")

print(result['score'])  # Score from 0 to 4
print(result['crack_times_display'])  # Human-readable estimates




2
{'online_throttling_100_per_hour': '62 years', 'online_no_throttling_10_per_second': '2 months', 'offline_slow_hashing_1e4_per_second': '2 hours', 'offline_fast_hashing_1e10_per_second': 'less than a second'}


In [27]:

def evaluate_passwords(password_list):
    """
    Takes a list of passwords and returns:
    - average score (0 to 4)
    - score distribution
    - crack time estimates
    """
    scores = []
    crack_times = []

    for pw in password_list:
        result = zxcvbn(pw)
        scores.append(result['score'])
        crack_times.append(result['crack_times_seconds']['offline_fast_hashing_1e10_per_second'])

    avg_score = sum(scores) / len(scores)
    score_distribution = {
        score: scores.count(score) for score in range(5)
    }

    return {
        'average_score': avg_score,
        'score_distribution': score_distribution,
        'avg_crack_time_secs': sum(crack_times) / len(crack_times)
    }



## Easy


In [28]:

# 🔧 Example usage
passwords = [
    "123456",
    "letmein",
    "password",
    "aaaaaaa",
    "987654",
]



In [29]:

result = evaluate_passwords(passwords)


In [30]:

print("Average score:", result['average_score'])
print("Score distribution:", result['score_distribution'])
print("Average crack time (seconds):", result['avg_crack_time_secs'])



Average score: 0.0
Score distribution: {0: 5, 1: 0, 2: 0, 3: 0, 4: 0}
Average crack time (seconds): 3.12E-9



## Medium


In [34]:

# 🔧 Example usage
passwords = [
    "123456",
    "letmein",
    "MyP@ssword123",
    "Tr0ub4dor&3",
    "5tr0ngP@ssw0rd!",
]


In [35]:

result = evaluate_passwords(passwords)


In [36]:


print("Average score:", result['average_score'])
print("Score distribution:", result['score_distribution'])
print("Average crack time (seconds):", result['avg_crack_time_secs'])




Average score: 1.8
Score distribution: {0: 2, 1: 0, 2: 1, 3: 1, 4: 1}
Average crack time (seconds): 2.0028453004



## Constraints generated and likely harder


In [37]:

for pass_generated in constraintsGeneratedPasswords:
    print(pass_generated)


z4SvfVncfS14
o$^&^k&73Gs1
d6fTX6vI^!sy
@A#UbUW7S$pz
Sf5I@n2$dejM
Vz$N1D1$xTPP
2TR*pw&@KfJJ
!PLcQ8xUdM%^


In [38]:

result = evaluate_passwords(constraintsGeneratedPasswords)


In [39]:


print("Average score:", result['average_score'])
print("Score distribution:", result['score_distribution'])
print("Average crack time (seconds):", result['avg_crack_time_secs'])



Average score: 4.0
Score distribution: {0: 0, 1: 0, 2: 0, 3: 0, 4: 8}
Average crack time (seconds): 100.0000000001


In [ ]:
remeber ll this ]